In [1]:
import os
import time
import argparse
import random
import numpy as np
import tensorflow as tf

from models.CTS import build_CTS
from configs.configs import Configs

from utils.read_data import read_essays, read_pos_vocab
from utils.general_utils import get_scaled_down_scores, pad_hierarchical_text_sequences, get_attribute_masks

from evaluators.multitask_evaluator_all_attributes import Evaluator as AllAttEvaluator

In [7]:
original_array1 = np.loadtxt("test1.txt")
new_array1 = np.zeros(len(original_array1), dtype="int")
for index, item in enumerate(original_array1):
    new_array1[index] = int(item)

original_array2 = np.loadtxt("test2.txt")
new_array2 = np.zeros(len(original_array2), dtype="float")
for index, item in enumerate(original_array2):
    new_array2[index] = int(item)

original_array3 = np.loadtxt("test3.txt")
new_array3 = np.zeros(len(original_array3), dtype="float")
for index, item in enumerate(original_array3):
    new_array3[index] = int(item)
    
configs = Configs()


In [8]:
model = build_CTS(36, 97, 50, 35, 51, configs, 9)
model.load_weights('saved_model/ckpt1')

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pos_word_input (InputLayer)     [(None, 4850)]       0                                            
__________________________________________________________________________________________________
pos_x (Embedding)               (None, 4850, 50)     1800        pos_word_input[0][0]             
__________________________________________________________________________________________________
pos_x_maskedout (ZeroMaskedEntr (None, 4850, 50)     0           pos_x[0][0]                      
__________________________________________________________________________________________________
pos_drop_x (Dropout)            (None, 4850, 50)     0           pos_x_maskedout[0][0]            
_______________________________________________________________________________________

In [9]:
ex = [new_array1, new_array2, new_array3]

In [10]:
import shap
from utils.read_data import text_tokenizer
import re

def custom_tokenizer(s, return_offsets_mapping=True):
        """ Custom tokenizers conform to a subset of the transformers API.
        """
        pos = 0
        offset_ranges = []
        input_ids = []
        for m in re.finditer(r"\W", s):
            start, end = m.span(0)
            offset_ranges.append((pos, start))
            input_ids.append(s[pos:start])
            pos = end
        if pos != len(s):
            offset_ranges.append((pos, len(s)))
            input_ids.append(s[pos:])
        out = {}
        out["input_ids"] = input_ids
        if return_offsets_mapping:
            out["offset_mapping"] = offset_ranges
        return out
    
    
masker = shap.maskers.Text(custom_tokenizer)
explainer = shap.Explainer(model, masker)

TypeError: shape() missing 1 required positional argument: 's'

In [ ]:
shap_values = explainer(ex)